In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import datetime
import os
import skimage
import random
from tensorflow.python.keras.utils.data_utils import Sequence
from scipy.ndimage import zoom
from scipy.ndimage import shift

In [2]:
#input variables
path = 'image_arrays_new_new\\'
validation_path = path + 'validation'
training_path = path + 'training'
test_path = path + 'test'
#model variables
batch_size = 30 #
epoch_number = 75
learning_rate = 1e-3 

params = {'dim': (64,64),
          'batch_size': batch_size,
          'n_classes': 2,
          'n_channels': 5,
          'shuffle': True}


#more parameters means more prone to overfitting, and I am 5/3 times worse on parameters compared to the paper I have
#based this on. (5 bands instead of 3) I need to find ways to add more regularization, or otherwise might try reducing my number
#of layers to reduce the number of parameters.

In [3]:
#https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/48097478
def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [4]:
#https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class DataGenerator(Sequence):

    def __init__(self, list_IDs, labels, batch_size=32, dim=(64,64), n_channels=3,
                 n_classes=2, shuffle=True):
     #   'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
    #'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
    #'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
    
      
      # Generate data and perform augmentation
        for i, ID in enumerate(list_IDs_temp):
            
          # Store sample
            X[i,] = np.load('image_arrays/' + ID + '.npy')
                          
            #flip
            if random.random() > 0.5:
                X[i,] = np.flip(X[i,],0)
            if random.random() > 0.5:
                X[i,] = np.flip(X[i,],1)
            
            #shift
            if random.random() > 0.5 :
                X[i,] = shift(X[i,], (4,0,0), mode='nearest')
            elif random.random() > 0.5 :
                X[i,] = shift(X[i,], (-4,0,0), mode='nearest')
                              
            if random.random() > 0.5 :
                X[i,] = shift(X[i,], (0,4,0), mode='nearest')
            elif random.random() > 0.5 :
                X[i,] = shift(X[i,], (0,-4,0), mode='nearest')
          
            #zoom in/out
            zoom_factor = random.uniform(0.75,1.3)
            X[i,] = clipped_zoom(X[i,],zoom_factor)
            
            #rotate
            angle = 45*random.random()
            X[i,] = skimage.transform.rotate(X[i,], angle=angle, mode='reflect')
            
            # Store class
            y[i] = self.labels[ID]
    
        if self.n_classes > 2:
            return X, keras.utils.to_categorical(y, num_classes=self.n_classes)
        else:
            return X, y

    def __len__(self):
    #'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
    #  'Generate one batch of data'
      # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

      # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

      # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

In [5]:
galaxyzoo = pd.read_csv("zoo2MainSpecz.csv", usecols=[8,15,21], nrows=10000)
Class = galaxyzoo["gz2class"].values
dictionary = {'A':int(2),'E':np.array([0]),'S':np.array([1])}
#resave using my dictionary
target = np.empty((len(Class)))
for i in range(len(Class)):
    target[i] = dictionary[Class[i][0]]
    
spiral_debiased = galaxyzoo["t01_smooth_or_features_a02_features_or_disk_debiased"].values
elliptical_debiased = galaxyzoo["t01_smooth_or_features_a01_smooth_debiased"].values

In [6]:
train_list = os.listdir(training_path)
train_list_new = list()
for i,file in enumerate(train_list):
    if (spiral_debiased[i] > 0.95) or (elliptical_debiased[i] > 0.90):
        train_list_new.append(file.split('.')[0])
val_list = os.listdir(validation_path)
val_list_new = list()
for i,file in enumerate(val_list):
    #if spiral_debiased[i] > 0.97 or elliptical_debiased[i] > 0.92:
    val_list_new.append(file.split('.')[0])

        
partition = {'train':train_list_new,'validation':val_list_new}

labels = {}
for i in range(10000):
    name = 'array_number_{}'.format(i)
    labels.update({name:target[i]})
    
print(len(train_list_new))
print(len(val_list_new))

1875
1119


In [7]:
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

In [8]:
#so this is pretty neat, you can create a keras callback to display on tensorboard using a simplified summary tf api

#and also this is an example of how to change the lr on the fly, which is pretty handy
#https://keras.io/callbacks/


"""
    file_writer = tf.summary.create_file_writer(logdir + "/metrics")
    file_writer.set_as_default()
"""
def lr_schedule(epoch,lr):

#Returns a custom learning rate that decreases as epochs progress.
    if epoch > 15:
        lr = 1e-4
    if epoch > 30:
        lr = 1e-5

    tf.summary.scalar('learning_rate', tensor=lr)
    return lr

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)

logdir="summaries/scalars/" + str(datetime.datetime.now().timestamp())
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,
                                                   histogram_freq=1,
                                                   write_graph=False,
                                                   write_grads=True,)
                                                   #write_images=True)
#will it still print stuff

In [9]:
def create_model(learning_rate=learning_rate):
    
    model = keras.Sequential([])
    
    model.add(keras.layers.Conv2D(input_shape=(64,64,5),filters=32,kernel_size=6,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.5))
    
    model.add(keras.layers.Conv2D(filters=64,kernel_size=5,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=2,))
    model.add(keras.layers.Dropout(rate=0.25)) #best = 0.25
    
    model.add(keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=2,))
    model.add(keras.layers.Dropout(rate=0.25)) #best = 0.25
    
    
    model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.25)) #best = 0.35

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=64,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(units=1,activation=tf.nn.sigmoid)) #tf.nn.softmax for categorical, sigmoid for binary
    
    adam = keras.optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy']) 
    return(model)

In [10]:
keras.backend.clear_session()
model = create_model(learning_rate = learning_rate)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
steps_to_take = int(len(os.listdir(training_path))/batch_size /6)
val_steps_to_take = int(len(os.listdir(validation_path))/batch_size)
                #typically be equal to the number of unique samples if your dataset
                #divided by the batch size.

print(steps_to_take)
print(val_steps_to_take)

34
37


In [12]:
#another callback
#reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-8, verbose=1, mode='min')

In [13]:
hist = model.fit_generator(generator=training_generator,
                    steps_per_epoch=steps_to_take, 
                    epochs=epoch_number,
                    validation_data=validation_generator,
                    validation_steps=val_steps_to_take,
                    verbose=2,
                    callbacks=[tensorboard_callback,lr_callback])

Instructions for updating:
Use tf.cast instead.


C:\Users\awe2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Epoch 1/75
37/37 [==============================] - 77s 2s/step - loss: 0.6642 - acc: 0.6153
 - 194s - loss: 0.6975 - acc: 0.5527 - val_loss: 0.6642 - val_acc: 0.6153
Epoch 2/75
37/37 [==============================] - 49s 1s/step - loss: 0.6463 - acc: 0.6216
 - 118s - loss: 0.6782 - acc: 0.5753 - val_loss: 0.6463 - val_acc: 0.6216
Epoch 3/75
37/37 [==============================] - 50s 1s/step - loss: 0.6845 - acc: 0.6162
 - 120s - loss: 0.6419 - acc: 0.6253 - val_loss: 0.6845 - val_acc: 0.6162
Epoch 4/75
37/37 [==============================] - 50s 1s/step - loss: 0.5187 - acc: 0.7640
 - 117s - loss: 0.5926 - acc: 0.6892 - val_loss: 0.5187 - val_acc: 0.7640
Epoch 5/75
37/37 [==============================] - 50s 1s/step - loss: 0.5961 - acc: 0.6667
 - 118s - loss: 0.5634 - acc: 0.7285 - val_loss: 0.5961 - val_acc: 0.6667
Epoch 6/75
37/37 [==============================] - 52s 1s/step - loss: 0.5049 - acc: 0.7631
 - 121s - loss: 0.5571 - acc: 0.7355 - val_loss: 0.5049 - val_acc: 0.763

Epoch 50/75
37/37 [==============================] - 51s 1s/step - loss: 0.4045 - acc: 0.8261
 - 120s - loss: 0.4012 - acc: 0.8371 - val_loss: 0.4045 - val_acc: 0.8261
Epoch 51/75
37/37 [==============================] - 51s 1s/step - loss: 0.3895 - acc: 0.8270
 - 121s - loss: 0.3844 - acc: 0.8269 - val_loss: 0.3895 - val_acc: 0.8270
Epoch 52/75
37/37 [==============================] - 50s 1s/step - loss: 0.3874 - acc: 0.8252
 - 120s - loss: 0.3963 - acc: 0.8285 - val_loss: 0.3874 - val_acc: 0.8252
Epoch 53/75
37/37 [==============================] - 51s 1s/step - loss: 0.3878 - acc: 0.8351
 - 120s - loss: 0.3868 - acc: 0.8280 - val_loss: 0.3878 - val_acc: 0.8351
Epoch 54/75
37/37 [==============================] - 50s 1s/step - loss: 0.3759 - acc: 0.8333
 - 119s - loss: 0.3706 - acc: 0.8468 - val_loss: 0.3759 - val_acc: 0.8333
Epoch 55/75
37/37 [==============================] - 52s 1s/step - loss: 0.3902 - acc: 0.8261
 - 121s - loss: 0.3853 - acc: 0.8403 - val_loss: 0.3902 - val_acc:

In [14]:
#test_loss, test_acc = model.evaluate(test_images, test_target)
#print('Test accuracy:', test_acc)
#print('Test loss:', test_loss)

In [15]:
#y_prob = model.predict(X)

In [16]:
#source list
"""
https://fizzylogic.nl/2017/05/08/monitor-progress-of-your-keras-based-neural-network-using-tensorboard/

https://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras

https://arxiv.org/pdf/1711.05744.pdf

https://arxiv.org/pdf/1807.00807.pdf

https://github.com/jameslawlor/kaggle_galaxy_zoo/blob/master/galaxy_zoo_keras.ipynb

https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

#https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/48097478

https://distill.pub/2018/building-blocks/ what I want to do with this after it is working.
"""

'\nhttps://fizzylogic.nl/2017/05/08/monitor-progress-of-your-keras-based-neural-network-using-tensorboard/\n\nhttps://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras\n\nhttps://arxiv.org/pdf/1711.05744.pdf\n\nhttps://arxiv.org/pdf/1807.00807.pdf\n\nhttps://github.com/jameslawlor/kaggle_galaxy_zoo/blob/master/galaxy_zoo_keras.ipynb\n\nhttps://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly\n\n#https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/48097478\n\nhttps://distill.pub/2018/building-blocks/ what I want to do with this after it is working.\n'

In [17]:
"""
model_1:

batch_size = 32 #
epoch_number = 50
learning_rate = 1e-3 

overfitted, but not unstable after 50 epochs unlike w/o augmentation.

augments= flips, rotations

tensorboard: 1561000349.947442

architecture:
model.add(keras.layers.Conv2D(input_shape=(64,64,5),filters=32,kernel_size=6,padding='same',activation=tf.nn.relu))
model.add(keras.layers.Dropout(rate=0.5))

model.add(keras.layers.Conv2D(filters=64,kernel_size=5,padding='same',activation=tf.nn.relu))
model.add(keras.layers.MaxPool2D(pool_size=2,))
model.add(keras.layers.Dropout(rate=0.25)) #best = 0.25

model.add(keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation=tf.nn.relu))
model.add(keras.layers.MaxPool2D(pool_size=2,))
model.add(keras.layers.Dropout(rate=0.25)) #best = 0.25

model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation=tf.nn.relu))
model.add(keras.layers.Dropout(rate=0.25)) #best = 0.35

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=64,activation=tf.nn.relu))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=2,activation=tf.nn.softmax))
"""

"""
model_2

batch_size = 32 #
epoch_number = 75
learning_rate = 5e-4 

same architecture as above

tensorboard: 1561040263.807743

premise: IDK lol just want something to run while I am working on my real job, but next time I think we will up dropouts,
then decrease layers if that still tends to overfit
"""

"""
model_3 

we get rid of the last convoluational layer, and increase all dropouts from 0.25 to 0.4

else the same.

Also, our steps to take was flipped from what they were meant to be; so we were training with very small epochs before.

This means that our network does not train at the same speed, lol

tensorboard = 1561056117.079296
"""

"""
model_4

tried reducing LR to 1e-4, cut the number of filters in the third convolutional layer to 64, and increased dropout to 0.4 on all
dropout = 0.25 layers. Newtwork accuracy did not increase, epoch accuracy convered lower than other attempts, newtork in general 
converged slower. BAD idea, maybe too little parameters. sweet spot might be in the middle somewhere. also did batch size => 64

trying to fix network to make better use of the fact that we are in binary mode. , changed loss to binary_crossentropy, changed
activation to sigmoid to match. reverted all other parameters to original archiecture, minus the fourth convolutional layer,
which I have shown is unneccessary, I think. (incorrect, the best performing model was with that convolutional layer., lr=5e-4)

tensorboard = 1561076766.823376

Performs about as well as the model_2, we will keep the binary since that is what the paper we based off did.

# params  = 2,334,817
"""

"""
model_5

We introduce zooming in or out into our data augmentation, and we will introduce the last layer back into the model, since that
was the best performing model.  Also am increasing dropout from 0.25 to 0.3 on layers, just to add a bit more regularization.

Next thing I can add if this doesnt work is the moving of the center pixel around, and then after that we might consider
increasing depth b/c I guess the best performing models have had more depth so far, but it is against intuition to do so.

This performed wrose than model_2, which is disheartening because there was more augmentation. (hint, I maybe doing augmentat-
ion wrong???)

tensorboard=1561130093.100813
"""

"""
model_6

Okay, eveything should be exactly the same as the paper we based off of. augmentations shifts of about 5% have been added, as
as scaling of each channel to themselves. "The flux values are normalised to the maximum value in each filter for each galaxy."

I probably didnt need to waste time and just implement their result fully in the first place, but hey whatever I am learning

Litteraly everything is the same as their paper except that we have (64,64) , we are not at the petrosian radius, and we have 5
channels. So I will probably mess with architecure from here. I may start with increasring depth since it is one thing I havent
tried yet.
"""

"""
model_idk anymore lol

tensorboard=scalars\1561343515.138241

added more layers, increased parameters. compared to model_2, the model didnt perform as well.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d (Conv2D)              (None, 64, 64, 32)        5792      
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        51264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       32896     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 256)       131328    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 512)         1180160   
_________________________________________________________________
dropout_4 (Dropout)          (None, 8, 8, 512)         0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                2097216   
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
=================================================================
Total params: 3,498,721
Trainable params: 3,498,721
Non-trainable params: 0
_________________________________________________________________

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-17-7db5de09873d>, line 157)